## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

application_clean=application_df.drop(columns="NAME")
application_clean2=application_clean.drop(columns="EIN")
application_clean3=application_clean.drop(columns="AFFILIATION")
application_df_clean = pd.DataFrame(application_clean3)
application_df_clean.head()

,EIN,APPLICATION_TYPE,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,T10,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,T3,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,T5,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,T3,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,T3,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column.
application_df_clean.nunique()

EIN                       34299
APPLICATION_TYPE             17
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts for binning
application_count = application_df_clean.value_counts("APPLICATION_TYPE")

In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_count[application_count <1000].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df_clean['APPLICATION_TYPE'] = application_df_clean['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df_clean['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df_clean.value_counts("CLASSIFICATION")
classification_count

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2190        1
C2380        1
C2500        1
C2561        1
C8210        1
Length: 71, dtype: int64

In [9]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df_clean.value_counts("CLASSIFICATION")
classification_counts_greater1 = classification_counts[classification_counts > 1]
classification_counts_greater1

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1278       10
C1238       10
C1235        9
C1237        9
C7210        7
C1720        6
C4100        6
C2400        6
C1600        5
C1257        5
C2710        3
C1260        3
C0           3
C1267        2
C1246        2
C1256        2
C3200        2
C1234        2
dtype: int64

In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#classifications_to_replace = ["C1000","C2000","C1200","C1235","C1237","C7210","C1720","C4100","C2400","C1600","C1257","C2710","C1260","C0","C1267","C1246","C1256","C3200","C1234","C4500","C4120","C5200","C3700","C6100","C4200","C1900","C2600","C1728","C1236","C1245","C1248","C1283","C1370","C1570","C1580","C1732","C2570","C1820","C2150","C2170","C2190","C2380","C2500","C2561","C8210"]
classifications_to_replace = classification_count[classification_count <50].index
# Replace in dataframe
for cls in classifications_to_replace:
    application_df_clean['CLASSIFICATION'] = application_df_clean['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df_clean['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      391
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
Name: CLASSIFICATION, dtype: int64

In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
#dummy_columns = ["APPLICATION_TYPE","CLASSIFICATION","AFFILIATION","USE_CASE","ORGANIZATION","STATUS","INCOME_AMT","SPECIAL_CONSIDERATIONS","IS_SUCCESSFUL"]
application_df_dummies = pd.get_dummies(application_df_clean)
application_df_dummies.head()

,EIN,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,10520599,1,5000,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,10531628,1,108590,1,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,10547893,1,5000,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,10553066,1,6692,1,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,10556103,1,142590,1,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [12]:
# Split our preprocessed data into our features and target arrays
y = application_df_dummies["IS_SUCCESSFUL"]
X = application_df_dummies.drop(columns="IS_SUCCESSFUL")
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

**Optimization Attempt #1**


- Dropping more or fewer columns. (dropped Affiliation column)
- Increasing or decreasing the number of values for each bin. (Changed the threshold for application types (1000) and classification columns (50))

## Compile, Train and Evaluate the Model

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(tf.keras.layers.Dense(units=70, activation='relu', input_dim=input_features))

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=32, activation='relu'))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn1.summary()

# Compile the model
nn1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
nn1.fit(X_train_scaled,y_train,epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 70)                3220      
                                                                 
 dense_1 (Dense)             (None, 32)                2272      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 5525 (21.58 KB)
Trainable params: 5525 (21.58 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.6222 - accuracy: 0.6462
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6096 - accuracy: 0.6583
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6059 - 

**Optimization Attempt #2**


- Add more neurons to a hidden layer.
- Add more hidden layers.

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=input_features))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=40, activation='relu'))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(units=20, activation='relu'))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn2.summary()

# Compile the model
nn2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
nn2.fit(X_train_scaled,y_train,epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                3680      
                                                                 
 dense_4 (Dense)             (None, 40)                3240      
                                                                 
 dense_5 (Dense)             (None, 20)                820       
                                                                 
 dense_6 (Dense)             (None, 1)                 21        
                                                                 
Total params: 7761 (30.32 KB)
Trainable params: 7761 (30.32 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100
804/804 [==============================] - 5s 4ms/step - loss: 0.6224 - accuracy: 0.6438
Epoch 2/100
804/804 [==============================

**Optimization Attempt #3**
- Use different activation functions for the hidden layers.
- Add or reduce the number of epochs to the training regimen.

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=80, activation='tanh', input_dim=input_features))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=40, activation='tanh'))

# Third hidden layer
nn3.add(tf.keras.layers.Dense(units=20, activation='tanh'))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn3.summary()

# Compile the model
nn3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
nn3.fit(X_train_scaled,y_train,epochs=200)

# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                3680      
                                                                 
 dense_4 (Dense)             (None, 40)                3240      
                                                                 
 dense_5 (Dense)             (None, 20)                820       
                                                                 
 dense_6 (Dense)             (None, 1)                 21        
                                                                 
Total params: 7761 (30.32 KB)
Trainable params: 7761 (30.32 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/200
804/804 [==============================] - 3s 2ms/step - loss: 0.6222 - accuracy: 0.6445
Epoch 2/200
804/804 [==============================

In [16]:
# Export our model to HDF5 file
nn3.save("AlphabetSoupCharity_Optimization.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


** Alphabet Soup Charity: Written Report **
** Data Preprocessing


*   What variable is the target for your model?
    The 'Is_Successful' variable is the target for this model
*   What variable(s) are the features for your model? The features for this model started as the variables in all columns other than 'Is_Succesful'. We also cleaned for non-benefitial columns ('EIN' and 'Name') to not be included as features. Also, as part of my first optimization attempt I removed 'Affiliation' from features.
* What variable(s) should be removed from the input data because they are neither targets nor features? As noted above, the non-beneficial columns identified included 'EIN', 'Name', and, later, 'Affiliation'.

**Compiling, Training and Evaluating the Model


*   How many neurons, layers, and activation functions did you select for your neural network model, and why? My main/ original model included 2 hidden layers and one output layer. I started with 70 neurons in the first hidden layer as there were 69 columns, and I understand that usually the number of nuerons should be a round number close to the number of features. My original model leveraged ReLu (Rectified Linear Unit) as the activation function for the first two layers as it is good for classifications and is the most commonly used activation function used in nueral networks. Then for the output layer I chose to use the Sigmoid function as this is also good for classifications.
*   Were you able to achieve the target model performance? No, but I was very close with my first model! This model was 72.5% accurate
*   What steps did you take in your attempts to increase model performance? I leveraged most of the suggested steps to increase model performance in one of my three optimization attempts.
      *   In Optimization 1, I dropped the Affiliation column and increased the values in the classification and application types bins.
      *   In Optimization 2, I added an additional ReLu layer and added 10 additional neurons.
      *   In Optimization 3, I changed the 3 ReLu layers to Tanh functions, which are good for classifications with multiple classes. I also increased the number of epochs from 100 to 200.

**Summary: The first model performed the best, with 72.5% accuracy. The optimimizations resulted in 66.9%, 66.5% and 62% accuracy. In another optimization, I would try starting from the details I had in the original model, trying a higher binning threshold for application types, classifications, and try cleaning a few other features that might not be necessary, such as maybe status and special considerations.

